# Agent Bot Demo

This notebook demonstrates a simple conversational agent using LangGraph and Ollama. The agent maintains conversation history and can save the dialogue to a file.


In [7]:
# Import required libraries
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
import os

In [ ]:
# Initialize LLM
llm = ChatOllama(model="llama3.2:latest")

# Verify LLM is loaded
assert llm is not None, "LLM failed to initialize"


# Define agent state type
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

In [ ]:
# Define core functions
def process(state: AgentState) -> AgentState:
    """Process response from LLM"""
    response = llm.invoke(state["messages"])
    print(f"\nAI: {response.content}")
    state["messages"].append(AIMessage(content=response.content))
    return state


def save_conversation(
    conversation_history: List[Union[HumanMessage, AIMessage]],
    filename: str = "chat_log.txt",
):
    """Save conversation history to a file"""
    with open(filename, "w") as file:
        file.write("Conversation history:\n")
        for m in conversation_history:
            if isinstance(m, HumanMessage):
                file.write(f"User: {m.content}\n")
            elif isinstance(m, AIMessage):
                file.write(f"AI: {m.content}\n\n")
        file.write("End of Conversation")
    print(f"Conversation saved to {filename}")

In [4]:
# Set up and compile the graph
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

In [ ]:
# Test the agent with sample conversations
test_examples = [
    {
        "messages": ["Hello, how are you?"],
        "exp_keywords": ["assist", "help", "language", "model"],
    },
    {
        "messages": ["What can you help me with?"],
        "exp_keywords": ["assist", "help", "support", "can"],
    },
]


def validate_response(response: str, keywords: list) -> tuple[bool, list]:
    """Validate that response contains expected keywords"""
    found_keywords = []
    for keyword in keywords:
        if keyword.lower() in response.lower():
            found_keywords.append(keyword)
    return len(found_keywords) >= 1, found_keywords


conversation_history = []

for example in test_examples:
    for message in example["messages"]:
        print(f"\nUser: {message}")
        conversation_history.append(HumanMessage(content=message))
        result = agent.invoke({"messages": conversation_history})
        conversation_history = result["messages"]

        # Get the last AI response
        response = conversation_history[-1].content

        # Basic response validation
        assert isinstance(
            conversation_history[-1], AIMessage
        ), "Invalid response format"
        assert len(response) > 0, "Empty response received"

        # Keyword validation
        is_valid, found = validate_response(response, example["exp_keywords"])
        assert is_valid, (
            f"Response missing expected keywords. Found: {found}. "
            f"Expected at least 1 from: {example['exp_keywords']}"
        )
        print(f"Found keywords: {found}")


User: Hello, how are you?

AI: I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?
Found keywords: ['assist', 'help', 'language', 'model']

User: What can you help me with?

AI: I can help with a wide range of topics and tasks. Here are some examples:

1. **Answering questions**: I can provide information on various subjects, including history, science, technology, literature, and more.
2. **Language translation**: I can translate text from one language to another.
3. **Writing assistance**: I can help with writing tasks such as suggesting alternative phrases, rephrasing sentences, and providing grammar and spelling corrections.
4. **Conversation**: I can engage in natural-sounding conversations, using context and understanding to respond to questions and statements.
5. **Brainstorming**: I can generate ideas and suggestions on a giv

In [10]:
# Save the test conversation
save_conversation(conversation_history, "test_conversation.txt")
assert os.path.exists("test_conversation.txt"), "Conversation file not saved"

Conversation saved to test_conversation.txt
